In [ ]:
import re
import sys
import numpy as np

powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2020 SP4\Python\3.8'
if powerfactory_path not in sys.path:
    sys.path.append(powerfactory_path)
import powerfactory as pf

try:
    from pfcommon import *
except:
    sys.path.append('..')
    from pfcommon import *

In [ ]:
app = pf.GetApplication()
if app is None:
    raise Exception('Cannot get PowerFactory application')
else:
    print('Successfully obtained PowerFactory application.')

In [ ]:
project_name = '\\Terna_Inerzia\\ENTSOE_20221018'
err = app.ActivateProject(project_name)
if err:
    raise Exception(f'Cannot activate project {project_name}')
print(f'Successfully activated project {project_name}.')

In [ ]:
generators = app.GetCalcRelevantObjects('*.ElmSym')
lines = app.GetCalcRelevantObjects('*.ElmLne')
buses = app.GetCalcRelevantObjects('*.ElmTerm')
loads = app.GetCalcRelevantObjects('*.ElmLod')
transformers = app.GetCalcRelevantObjects('*.ElmTr2')
shunts = app.GetCalcRelevantObjects('*.ElmShnt')
capacitors = app.GetCalcRelevantObjects('*.ElmScap')
impedances = app.GetCalcRelevantObjects('*.ElmZpu')
n_generators = len(generators)
n_lines, n_buses, n_loads, n_transformers = len(lines), len(buses), len(loads), len(transformers)
n_shunts, n_capacitors, n_impedances = len(shunts), len(capacitors), len(impedances)
print(f'There are {n_generators} generators.')
print(f'There are {n_lines} lines.')
print(f'There are {n_buses} buses.')
print(f'There are {n_loads} loads.')
print(f'There are {n_transformers} transformers.')
print(f'There are {n_shunts} shunts.')
print(f'There are {n_capacitors} series capacitors.')
print(f'There are {n_impedances} common impedances.')

In [ ]:
load_flow = app.GetFromStudyCase('ComLdf')
err = load_flow.Execute()

get_name = lambda elem: elem.name

powergenerators = sorted([PowerGenerator(gen) for gen in generators], key=get_name)
powerloads = sorted([PowerLoad(load) for load in loads], key=get_name)
powerbuses = sorted([PowerBus(bus) for bus in buses], key=get_name)
powertransformers = sorted([PowerTransformer(transformer) for transformer in transformers], key=get_name)
powerlines = sorted([PowerLine(line) for line in lines], key=get_name)
powershunts = sorted([Shunt(shunt) for shunt in shunts], key=get_name)
powercapacitors = sorted([SeriesCapacitor(cap) for cap in capacitors], key=get_name)
powerimpedances = sorted([CommonImpedance(imp) for imp in impedances], key=get_name)

In [ ]:
with open('ENTSOE.inc', 'w') as fid:
    for gen in powergenerators:
        fid.write(str(gen) + '\n')
    fid.write('\n')
    for load in powerloads:
        fid.write(str(load) + '\n')
    fid.write('\n')
    for line in powerlines:
        fid.write(str(line) + '\n')
    fid.write('\n')
    for trans in powertransformers:
        fid.write(str(trans) + '\n')
    fid.write('\n')
    for bus in powerbuses:
        fid.write(str(bus) + '\n')
    fid.write('\n')
    for shunt in powershunts:
        fid.write(str(shunt) + '\n')
    fid.write('\n')
    for cap in powercapacitors:
        fid.write(str(cap) + '\n')
    fid.write('\n')
    for imp in powerimpedances:
        fid.write(str(imp) + '\n')